## Linear Regression algorithm

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

Data loading: 

In [2]:
boston_data = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston_data.data, boston_data.target, random_state=1234)

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

sklearn implementation:

In [4]:
model = LinearRegression()

model.fit(X_train, y_train)
predictions = model.predict(X_test)

print('r2 =', r2_score(y_test, predictions))

r2 = 0.7325732922440751


The simplest implementation: 

In [5]:
class SimpleLinearRegression:
    
    def fit(self, features, target):
        X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
        y = target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features):
        return features.dot(self.w) + self.w0

In [6]:
model = SimpleLinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print('r2 =', r2_score(y_test, predictions))

r2 = 0.7325732922440751


Linear regression with stochastic gradient descent + l2 regularization:

In [17]:
class SGDLinearRegression:
    def __init__(self, step_size, epochs, batch_size, reg_weight):
        self.step_size = step_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.reg_weight = reg_weight
    
    def fit(self, features, target):
        features = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)        
        w = np.zeros(features.shape[1])
        
        for _ in range(self.epochs):
            batches_count = features.shape[0] // self.batch_size
            for i in range(batches_count):
                start = i * self.batch_size
                end = (i + 1) * self.batch_size
                X_batch = features[start:end, :]
                y_batch = target[start:end]
                
                gradient = 2 * X_batch.T.dot(X_batch.dot(w) - y_batch) / X_batch.shape[0]
                reg = 2 * w.copy()
                reg[0] = 0
                gradient += self.reg_weight * reg
                
                w -= self.step_size * gradient

        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features):
        return features.dot(self.w) + self.w0

In [18]:
model = SGDLinearRegression(0.1, 20, 100, 0.001)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

r2 = r2_score(y_test, predictions)

print('r2 =', r2_score(y_test, predictions))

r2 = 0.7570253297465019
